In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()




In [35]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def laod_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = (
    {   "chat_history": laod_memory,
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)




In [36]:
invoke_chain("Is Aaronson guilty?")

content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [37]:
print(laod_memory(_))

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')]


In [38]:
invoke_chain("What message did he write in the table?")

content='He wrote "2+2=5" in the dust on the table.'


In [39]:
invoke_chain("Who is Julia?")

content='Julia is a character who was involved with Aaronson in the text.'


In [40]:
invoke_chain("What is my first question?")

content="Your first question was about Aaronson's guilt."


In [41]:
print(laod_memory(_))

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='He wrote "2+2=5" in the dust on the table.'), HumanMessage(content='Who is Julia?'), AIMessage(content='Julia is a character who was involved with Aaronson in the text.'), HumanMessage(content='What is my first question?'), AIMessage(content="Your first question was about Aaronson's guilt.")]
